<a href="https://colab.research.google.com/github/aicuai/Book-SD-MasterGuide/blob/main/ComfyJapan_Flux2_Wan22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このノートブックへの短縮URLは https://j.aicu.ai/CJCF2  原作はざすこさんの こちら↓　ComfyJapan勉強会への参加は[こちら](https://j.aicu.ai/ComfyJapan)

🍀ComfyUI_起動ノート（Wan2_2_フルカスタム版）🌐Drive自動保存付き_20251113.ipynb

# 🥚各種モデルのDL ＆ ComfyUIの起動🐣
##＜はじめる前の準備＞※重要※
### ❶生成に必要な各種モデルファイルのDL用URLを格納する各フォルダの記入欄に記載して下さい。
└🌐GoogleColab上の 📂ComfyUI / models  以下に、生成に必要な各種ファイルをDLします。（DL容量が少ない程起動が速くなります。）
### ❷（▶）ボタンをクリックしてComfyUIを起動


In [ ]:
# @markdown # 👈この（▶）ボタンをクリックしてComfyUIを起動（☕起動まで約5～6分）
# @markdown ### ※停止と再起動もこのボタンで行います。
# @markdown ## 🌱実行ログに青字で👀「https～ localhost:8188」が表示されたら🖱️クリックしてください。

!pip install torch==2.8.0 torchvision==0.23.0

# using_L4_GPU = True # @param {type:"boolean"}
using_L4_GPU = False
include_manager = True # @param {type:"boolean"}

# ==========================================
# 🔑 シークレットキーの読み込み
# ==========================================
from google.colab import userdata
import os

print("=" * 70)
print("🔑 Checking Secrets...")

# Hugging Face Token
try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        print("✅ HF_TOKEN found in Secrets")
        # 環境変数にもセット（huggingface-cli用）
        os.environ['HF_TOKEN'] = hf_token
    else:
        hf_token = ""
        print("⚠️ HF_TOKEN not found. Gated models (Flux.2 etc.) might fail.")
except Exception:
    hf_token = ""
    print("⚠️ HF_TOKEN not found in Secrets.")

# Civitai API Key
try:
    civitai_key = userdata.get('CIVITAI_KEY')
    if civitai_key:
        print("✅ CIVITAI_KEY found in Secrets")
    else:
        civitai_key = ""
except Exception:
    civitai_key = ""

print("=" * 70)


# ==========================================
# 📁 Google Drive マウント
# ==========================================
use_google_drive = False  # @param {type:"boolean"}

if use_google_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted")
else:
    print("⏸️  Google Drive not used")


# ==========================================
# 📤 Google Drive 出力設定
# ==========================================
enable_gdrive_output = False  # @param {type:"boolean"}
GDRIVE_OUTPUT = "/content/drive/MyDrive/ComfyUI_output"  # @param {type:"string"}

if use_google_drive and enable_gdrive_output:
    from pathlib import Path
    Path(GDRIVE_OUTPUT).mkdir(parents=True, exist_ok=True)
    print(f"✅ Output set to: {GDRIVE_OUTPUT}")


%cd /content
from IPython.display import clear_output
clear_output()

# Install dependencies
!pip install -q torchsde einops diffusers accelerate xformers==0.0.32.post1 triton==3.4
!pip install av spandrel albumentations onnx opencv-python onnxruntime color-matcher
!pip install --upgrade "protobuf>=5.29.1"
!pip install onnxruntime-gpu -y
# Install huggingface_hub CLI for reliable downloads
!pip install -U "huggingface_hub[cli]"

!git clone https://github.com/comfyanonymous/ComfyUI
!pip install -r /content/ComfyUI/requirements.txt
clear_output()

# Custom Nodes
%cd /content/ComfyUI/custom_nodes
if include_manager:
    !git clone https://github.com/ltdrdata/ComfyUI-Manager

!git clone --branch forQwen https://github.com/Isi-dev/ComfyUI_GGUF.git
!git clone https://github.com/Isi-dev/ComfyUI_DeleteModelPassthrough.git
!git clone https://github.com/Isi-dev/comfyui_controlnet_aux
!git clone https://github.com/kijai/ComfyUI-WanVideoWrapper
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/kijai/ComfyUI-segment-anything-2
!git clone https://github.com/kijai/ComfyUI-Florence2
!git clone https://github.com/john-mnz/ComfyUI-Inspyrenet-Rembg.git
!git clone https://github.com/Isi-dev/ComfyUI_Animation_Nodes_and_Workflows
!git clone https://github.com/rgthree/rgthree-comfy.git
!git clone https://github.com/cubiq/ComfyUI_essentials.git
!git clone https://github.com/chrisgoringe/cg-use-everywhere.git
!git clone https://github.com/aria1th/ComfyUI-LogicUtils.git
!git clone https://github.com/GACLove/ComfyUI-VFI.git

# Install requirements for custom nodes
import os
for root, dirs, files in os.walk("/content/ComfyUI/custom_nodes"):
    if "requirements.txt" in files:
        !pip install -r {os.path.join(root, "requirements.txt")}

# Fix protobuf
!pip install protobuf==5.29.1
clear_output()

# Apt packages
!apt-get -y install -qq aria2

%cd /content/ComfyUI

import subprocess
import re
from urllib.parse import urlparse

# ========================================
# 📥 ダウンロード関数群
# ========================================

def download_with_aria2c(link, folder):
    filename = link.split("/")[-1].split('?')[0]
    subprocess.run(['aria2c', '--console-log-level=error', '-c', '-x', '16', '-s', '16', '-k', '1M', '-d', folder, '-o', filename, link], check=True)

def download_civitai_model(link, token, folder):
    try:
        model_id = link.split("/models/")[1].split("?")[0]
    except:
        print(f"Skipping invalid Civitai link: {link}")
        return

    api_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
    if token:
        api_url += f"&token={token}"

    # Use wget for Civitai to handle redirects properly
    subprocess.run(['wget', '--content-disposition', '--show-progress', '-P', folder, api_url], check=False)

def parse_hf_url(url):
    # Extract repo_id and filename from URL
    # Format: https://huggingface.co/Repo/ID/blob/branch/path/to/file
    try:
        path_parts = url.replace("https://huggingface.co/", "").split("/")
        repo_id = f"{path_parts[0]}/{path_parts[1]}"

        # Find where 'blob' or 'resolve' or 'tree' is
        if "blob" in path_parts:
            idx = path_parts.index("blob")
        elif "resolve" in path_parts:
            idx = path_parts.index("resolve")
        elif "tree" in path_parts:
            idx = path_parts.index("tree")
        else:
            return None, None

        # filename is everything after branch (idx + 1 is branch)
        filename = "/".join(path_parts[idx+2:])
        return repo_id, filename
    except:
        return None, None

def download_huggingface_cli(url, folder):
    """
    Hugging Face CLIを使ってダウンロードする（Xet/LFS対応、認証対応）
    """
    repo_id, filename = parse_hf_url(url)
    if not repo_id or not filename:
        print(f"⚠️ Could not parse HF URL: {url} -> Fallback to aria2c")
        download_with_aria2c(url, folder)
        return

    print(f"⬇️ HF CLI Download: {filename} from {repo_id}")

    # huggingface-cli download repo_id filename --local-dir folder --local-dir-use-symlinks False
    # Token is automatically picked up from os.environ['HF_TOKEN']
    cmd = [
        "huggingface-cli", "download",
        repo_id,
        filename,
        "--local-dir", folder,
        "--local-dir-use-symlinks", "False",
        "--quiet"
    ]
    try:
        subprocess.run(cmd, check=True)
        print("   ✅ Done.")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Failed to download with CLI. Error: {e}")


def process_urls(url_list_str, folder):
    urls = [line.strip() for line in url_list_str.split('\n') if line.strip() and not line.startswith('#')]
    if not urls: return

    os.makedirs(folder, exist_ok=True)
    print(f"\n📂 Processing folder: {folder}")

    for url in urls:
        if "huggingface.co" in url:
            # Hugging Face URL -> Use CLI
            download_huggingface_cli(url, folder)
        elif "civitai.com" in url:
            # Civitai URL -> Use wget with Token
            download_civitai_model(url, civitai_key, folder)
        else:
            # Other -> aria2c
            download_with_aria2c(url, folder)


# ========================================
# 📥 モデル設定 (ユーザー入力エリア)
# ========================================

# @markdown ## 📁 diffusion_models フォルダ
diffusion_models_urls = """
https://huggingface.co/Comfy-Org/flux2-dev/blob/main/split_files/diffusion_models/flux2_dev_fp8mixed.safetensors
https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf
https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf
""" # @param {type:"string"}

# @markdown ## 📁 text_encoders フォルダ
# @markdown ※ここが18GBの問題ファイル。CLIダウンロードなら成功します。
text_encoders_urls = """
https://huggingface.co/Comfy-Org/flux2-dev/blob/main/split_files/text_encoders/mistral_3_small_flux2_fp8.safetensors
https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors
""" # @param {type:"string"}

# @markdown ## 📁 vae フォルダ
vae_urls = """
https://huggingface.co/Comfy-Org/flux2-dev/blob/main/split_files/vae/flux2-vae.safetensors
https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors
https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors
""" # @param {type:"string"}

# @markdown ## 📁 clip_vision フォルダ
clip_vision_urls = """
https://huggingface.co/Isi99999/Wan_Extras/resolve/main/clip_vision_h.safetensors
""" # @param {type:"string"}

# @markdown ## 📁 loras フォルダ
loras_urls = """
https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16.safetensors
https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors
""" # @param {type:"string"}

# @markdown ## 📁 sam2 フォルダ
sam2_urls = "https://huggingface.co/Kijai/sam2-safetensors/resolve/main/sam2.1_hiera_small.safetensors" # @param {type:"string"}

# @markdown ## 📁 checkpoints フォルダ
checkpoints_urls = "https://huggingface.co/bluepen5805/mellow_pencil-XL/resolve/main/mellow_pencil-XL-v1.0.0-base.safetensors" # @param {type:"string"}

# 実行
print("=" * 70)
print("🚀 Starting Smart Download...")
print("=" * 70)

process_urls(diffusion_models_urls, "/content/ComfyUI/models/diffusion_models")
process_urls(text_encoders_urls, "/content/ComfyUI/models/text_encoders")
process_urls(vae_urls, "/content/ComfyUI/models/vae")
process_urls(clip_vision_urls, "/content/ComfyUI/models/clip_vision")
process_urls(loras_urls, "/content/ComfyUI/models/loras")
process_urls(sam2_urls, "/content/ComfyUI/models/sam2")
process_urls(checkpoints_urls, "/content/ComfyUI/models/checkpoints")

print("=" * 70)
print("✓ All downloads completed!")


# ========================================
# ComfyUI起動
# ========================================
use_cloudflare = False # @param {type:"boolean"}

output_dir_arg = ""
if use_google_drive and enable_gdrive_output:
    output_dir_arg = f"--output-directory {GDRIVE_OUTPUT}"

import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print(f"\n✅ ComfyUI launched! Open the link below:\n")

    if use_cloudflare:
        p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in p.stderr:
            l = line.decode()
            if "trycloudflare.com " in l:
                print("🔗 Cloudflare URL:", l[l.find("http"):], end='')
    else:
        from google.colab import output
        output.serve_kernel_port_as_window(port)

if use_cloudflare:
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

cmd = f"python main.py {output_dir_arg} --dont-print-server"
if using_L4_GPU:
    cmd += " --cache-none"

!{cmd}

/content/ComfyUI
🚀 Starting Smart Download...

📂 Processing folder: /content/ComfyUI/models/diffusion_models
⬇️ HF CLI Download: split_files/diffusion_models/flux2_dev_fp8mixed.safetensors from Comfy-Org/flux2-dev
   ✅ Done.
⬇️ HF CLI Download: HighNoise/Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf from QuantStack/Wan2.2-I2V-A14B-GGUF
   ✅ Done.
⬇️ HF CLI Download: LowNoise/Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf from QuantStack/Wan2.2-I2V-A14B-GGUF
   ✅ Done.

📂 Processing folder: /content/ComfyUI/models/text_encoders
⬇️ HF CLI Download: split_files/text_encoders/mistral_3_small_flux2_fp8.safetensors from Comfy-Org/flux2-dev
   ✅ Done.
⬇️ HF CLI Download: split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors from Comfy-Org/Wan_2.1_ComfyUI_repackaged
   ✅ Done.

📂 Processing folder: /content/ComfyUI/models/vae
⬇️ HF CLI Download: split_files/vae/flux2-vae.safetensors from Comfy-Org/flux2-dev
   ✅ Done.
⬇️ HF CLI Download: split_files/vae/wan_2.1_vae.safetensors from Comfy-Org/Wan_2.2_Comf

<IPython.core.display.Javascript object>

FETCH ComfyRegistry Data: 10/108
FETCH ComfyRegistry Data: 15/108
FETCH ComfyRegistry Data: 20/108
FETCH ComfyRegistry Data: 25/108
FETCH ComfyRegistry Data: 30/108
FETCH ComfyRegistry Data: 35/108
FETCH ComfyRegistry Data: 40/108
FETCH ComfyRegistry Data: 45/108
FETCH ComfyRegistry Data: 50/108
FETCH ComfyRegistry Data: 55/108
FETCH ComfyRegistry Data: 60/108
FETCH ComfyRegistry Data: 65/108
FETCH ComfyRegistry Data: 70/108
FETCH ComfyRegistry Data: 75/108
FETCH ComfyRegistry Data: 80/108
FETCH ComfyRegistry Data: 85/108
FETCH ComfyRegistry Data: 90/108
FETCH ComfyRegistry Data: 95/108
FETCH ComfyRegistry Data: 100/108
FETCH ComfyRegistry Data: 105/108
FETCH ComfyRegistry Data [DONE]
[ComfyUI-Manager] default cache updated: https://api.comfy.org/nodes
FETCH DATA from: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/custom-node-list.json [DONE]
[ComfyUI-Manager] All startup tasks have been completed.
[DEPRECATION WARNING] Detected import of deprecated legacy API: /exten

# FLUX2

https://comfyanonymous.github.io/ComfyUI_examples/flux2/

# 🌱📋各種モデル_DL元リポジトリのメモ🌱
###　下記リンクのリポジトリから必要なモデルファイルのダウンロードURLを取得してきてください。
---
#🎬Wan2.2用 Diffusion Model
##＜ti2v‗5B‗fp16＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- wan2.2_ti2v_5B_fp16：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_ti2v_5B_fp16.safetensors)

##＜t2v‗14B‗fp8＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- Wan2.2-T2V-A14B(High‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors)
- Wan2.2-T2V-A14B(Low‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors)
##＜i2v‗14B‗fp8＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- Wan2.2-I2V-A14B(High‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors)
- Wan2.2-I2V-A14B(Low‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors)

##＜t2v‗GGUF＞ ※ModelLoaderの変更が必要
####※下記リンクをそのまま使用可能（GGUF_Q8）
- Wan2.2-T2V-A14B-HighNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-T2V-A14B-HighNoise-Q8_0.gguf)
- Wan2.2-T2V-A14B-LowNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-T2V-A14B-LowNoise-Q8_0.gguf)

####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-T2V-A14B-GGUF：https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/tree/main
##＜i2v‗GGUF＞ ※ModelLoaderの変更が必要
####※下記リンクをそのまま使用可能（GGUF_Q8）
- Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf)
- Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf)

####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-I2V-A14B-GGUF：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/tree/main)

##＜Fun&Inp＞ ※動画の人物の動きを再現
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-Fun＆Inpl：(https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/tree/main/Fun)

##＜Animate‗GGUF＞ ※動画中の被写体をすげ替えor参照画像に反映
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-Animate-A14B-GGUF(https://huggingface.co/QuantStack/Wan2.2-Animate-14B-GGUF/tree/main)

#🎬Wan2.2用LoRA
##＜lightx2v_4steps（高速化）＞※High＆Low 別々に指定
####※下記リンクをそのまま使用可能
- LightX2v_t2v-4step_High：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_t2v_lightx2v_4steps_lora_v1.1_high_noise.safetensors)
- LightX2v_t2v-4step_Low：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_t2v_lightx2v_4steps_lora_v1.1_low_noise.safetensors)
- LightX2v_i2v-4step_High：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors)
- LightX2v_i2v-4step_Low：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors)
##＜lightx2v_Rank（高速化）＞※High＆Low共通
####※下記リンクをそのまま使用可能
- lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16：(https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors)
- lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16：(https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16.safetensors)

- LightX2v_t2v&i2v-Rank：(https://huggingface.co/Kijai/WanVideo_comfy/tree/main/Lightx2v)

##＜relight_LoRA（Animate用）＞ ※Wan2.2 Animateに必須のLoRA
####※下記リンクをそのまま使用可能
- Wan2.2_relight：(https://huggingface.co/Kijai/WanVideo_comfy/tree/main/LoRAs/Wan22_relight)

#🎬Wan2.2用VAE
##＜VAE＞
####※下記リンクをそのまま使用可能
- Wan2.1_VAE.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors)
- wan2.2_vae.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan2.2_vae.safetensors)

#🎬Wan2.2用TextEncoder
##＜TextEncoder＞
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- umt5_xxl_fp16.safetensors：
(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/tree/main/split_files/text_encoders)

#🎬Wan2.2用ClipVision
##＜ClipVision＞
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- clip_vision_h.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/tree/main/split_files/clip_vision)

#🖼 StableDiffsions XL
##＜Checkpoint＞
Sierunami.v1：(https://civitai.com/api/download/models/1176276)
※CIVITAIのAPIキーが必要

mellow_pencil-XL-v1.0.0-base：(https://huggingface.co/bluepen5805/mellow_pencil-XL/resolve/main/mellow_pencil-XL-v1.0.0-base.safetensors)

mellow_pencil-XL-v1.0.0-chaos：(https://huggingface.co/bluepen5805/mellow_pencil-XL/resolve/main/mellow_pencil-XL-v1.0.0-chaos.safetensors)

##＜VAE＞
####※下記リンクをそのまま使用可能
sdxl_vae：(https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors)

---
